In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import pandas as pd
import re
import time
from transformers import pipeline

In [ ]:


# ✅ Credentials
SPOTIFY_CLIENT_ID = "968e5a07e9c74a74b2bf528d18f4c870"
SPOTIFY_CLIENT_SECRET = "8439583339f54d13936b71bf490e4fa2"
GENIUS_API_KEY = "ojzPuhVHc0uUz-6Usqc2SGeqe03s7tpURJ5FCb_QBjs4lEMRfcWyz4F6caQdDxIj"

# ✅ Initialize Genius client
genius = lyricsgenius.Genius(GENIUS_API_KEY, timeout=15, retries=3)

# ✅ Initialize theme classifier
theme_classifier = pipeline("zero-shot-classification", 
                          model="facebook/bart-large-mnli")
CANDIDATE_THEMES = [
    "romance", "sadness", "nature", "friendship",
    "self-love", "meditation", "adventure", "nostalgia",
    "party", "empowerment", "calm", "energy"
]

def clean_lyrics(lyrics):
    """Remove section headers and clean text"""
    return re.sub(r"\[.*?\]", "", lyrics).strip()

def get_lyrics(track_name, artist_name):
    """Get cleaned lyrics using lyricsgenius"""
    try:
        song = genius.search_song(track_name, artist_name)
        return clean_lyrics(song.lyrics) if song else None
    except Exception as e:
        print(f"Error getting lyrics for {track_name}: {str(e)}")
        return None

def analyze_themes(lyrics):
    """Analyze lyrics for themes using zero-shot classification"""
    if not lyrics:  
        return []
    return theme_classifier(lyrics[:1000], CANDIDATE_THEMES)["labels"][:3]

def main():
    # ✅ Load merged CSV file
    df = pd.read_csv("supabase_user_playlist_features.csv")  # Ensure the correct file path
    df["lyrics"] = None
    df["themes"] = None

    songs_data = []

    for index, row in df.iterrows():
        track_name = row["track_name"]
        artist_name = row["artist_name"]
        playlist_id = row["playlist_id"]

        print(f"Processing: {track_name} - {artist_name}")

        # ✅ Get lyrics and analyze themes
        lyrics = get_lyrics(track_name, artist_name)
        themes = analyze_themes(lyrics) if lyrics else []

        # ✅ Store results
        songs_data.append({
            "track_id": playlist_id,
            "track_name": track_name,
            "artist_name": artist_name,
            "lyrics": lyrics,
            "themes": themes
        })

        time.sleep(1)  # To avoid API rate limits

    # ✅ Convert to DataFrame and save
    lyrics_df = pd.DataFrame(songs_data)
    lyrics_df.to_csv("merged_Csv_with_lyrics.csv", index=False)
    print(f"Created database with {len(lyrics_df)} songs with lyrics and themes.")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("merged_csv_with_lyrics.csv")

# Remove duplicate rows (based on all columns)
df_cleaned = df.drop_duplicates(subset=["playlist_id","track_name"])

# OR remove duplicates based on specific columns (e.g., track_name and artist_name)
# df_cleaned = df.drop_duplicates(subset=["track_name", "artist_name"])

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv("lyrics_user_playlist.csv", index=False)

In [ ]:
print(f"Original rows: {len(df)}, Cleaned rows: {len(df_cleaned)}")

In [ ]:
df_user_features = pd.read_csv("supabase_user_playlist_features.csv")

# Remove duplicate rows (based on all columns)
df_user_features_cleaned = df_user_features.drop_duplicates(subset=["track_id","track_name"])

# OR remove duplicates based on specific columns (e.g., track_name and artist_name)
# df_cleaned = df.drop_duplicates(subset=["track_name", "artist_name"])

# Save the cleaned DataFrame to a new CSV file
df_user_features_cleaned.to_csv("user_playlist_features_cleaned.csv", index=False)

In [ ]:
print(f"Original rows: {len(df)}, Cleaned rows: {len(df_cleaned)}")